In [16]:
import numpy as np
import pprint
import pandas as pd
from scipy.spatial import distance
from collections import OrderedDict 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import  confusion_matrix,accuracy_score,classification_report
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [17]:
def gini_equation(class_0, class_1):  
    return (1 - (class_0/(class_0+class_1))**2 - (class_1/(class_0+class_1))**2)

In [18]:
def gini_ind(train_features, train_label, current_feature, current_index,indices):
    gini_value = 1.0
    left_0 , left_1, right_0,right_1 = 0,0,0,0
    
    for i in range(len(train_features)):
        if i not in indices:
            continue
        if train_features[current_index,current_feature]>train_features[i,current_feature] and train_label[i]==0:
            left_0 +=1
        elif train_features[current_index,current_feature]>train_features[i,current_feature] and train_label[i]==1:
            left_1 +=1
        elif train_features[current_index,current_feature]<=train_features[i,current_feature] and train_label[i]==0:
            right_0 +=1
        else:
            right_1 +=1
   
            
    D = left_0+left_1+right_0+right_1
    D1 = (left_0+left_1) / D
    D2 = (right_0+right_1)/D
    
    if ((left_0+left_1) != 0 and (right_0+right_1) != 0): 
        Gini_D1,Gini_D2 = gini_equation(left_0, left_1),gini_equation(right_0, right_1)
        gini_value = D1*Gini_D1 + D2*Gini_D2
    
    return gini_value

In [19]:
def divide_values(train_inp,train_lab,r,f,ind):
    left_node,left_class,right_node, right_class= [],[],[],[]
    
    for i in range(train_inp.shape[0]):
        if (train_inp[i,f]<train_inp[r,f] and i in ind):
            left_node.append(i)
            left_class.append(train_lab[i])
        elif i in ind:
            right_node.append(i)
            right_class.append(train_lab[i])
      
    return (left_node,left_class,right_node, right_class)

In [20]:
def best_split(train_input,train_output,index):
    gini = 1
    m,n = 0,0
    
    for i in range(train_input.shape[1]):
        for j in range(train_input.shape[0]):
            if j in index:
                g = gini_ind(train_input, train_output,i, j ,index) 
                if g <gini:
                    gini = g
                    m,n = j,i
    
    return m,n

In [21]:
def recursive_tree(input_list,class_list,feature_train,label_train,row_ind,tree_depth,ver):
    
    if len(input_list)==0 or len(class_list)>1:
        if len(input_list)==0:
            return label_train[row_ind]
        elif tree_depth==0:
            return Counter(class_list).most_common(1)[0][0] 
        else:
            return partition(feature_train, label_train, input_list, tree_depth-1)        
    else:
        return list(class_list)[0]
    
    return 

In [22]:
def partition(train_feature, train_label, indices, max_depth):
    
     
    if len(indices)==0:
        return
    indices = Counter(indices)
    row_index, feature_index = best_split(train_feature,train_label,indices)
    left,left_labels,right,right_labels = divide_values(train_feature,train_label,row_index,feature_index,indices)
             

    vertice = {}
    vertice['value'] = train_feature[row_index,feature_index]
    vertice['feature'] = feature_index
    
    left_values = set(left_labels)
    right_values = set(right_labels)
 
    vertice['left'] = recursive_tree(left,left_values,train_feature,train_label,row_index,max_depth, vertice)
    vertice['right'] = recursive_tree(right,right_values,train_feature,train_label,row_index,max_depth, vertice)
    

    return vertice
    

In [23]:
def find_test_value(test_input, head_tree):
    entry = test_input
    if head_tree==None:
        return -1
    f = head_tree['feature']
        
    
    if (entry[f]<head_tree['value'])  and (head_tree['left']==0 or head_tree['left']==1):
        return head_tree['left']
    elif (entry[f]>=head_tree['value'] ) and (head_tree['right']==0 or head_tree['right']==1):
        return head_tree['right']
    elif entry[f]<head_tree['value']:
        return find_test_value(test_input, head_tree['left'])
    else:
        return find_test_value(test_input, head_tree['right'])
                      
    return -1

In [24]:
def test_output(test_input, head):
    test_label = np.zeros(test_input.shape[0])
    for i in range(len(test_input)):
        test_label[i] = find_test_value(test_input[i],head)
        
    return test_label
    

In [25]:
def decision_tree(x_train, y_train, x_test,depth):
    head_node = partition(x_train,y_train,range(x_train.shape[0]),depth)
    test_class =  test_output(x_test,head_node)
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(head_node)                       
    return test_class

In [26]:
# for one data
data  = pd.read_csv('project3_dataset4.txt',sep='\t',header=None)
data.head()
data[0] = data[0].map( {'sunny':0, 'overcast':1, 'rain':2})
data[1] = data[1].map({'hot':0,'mild':1,'cool':2})
data[2] = data[2].map({'high':0,'normal':1})
data[3]= data[3].map({'weak':0,'strong':1})
labels = data.iloc[:,-1].values
features = data.iloc[:,:-1].values
d = 4
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2,random_state=42)
y_pred = decision_tree(X_train, Y_train, X_test,d)
print('accuracy',accuracy_score(Y_test,y_pred))
p,r,f,s = score(Y_test,y_pred)
print('precision:',p.mean())
print('recall:',r.mean())
print('f1measure',f.mean())

{   'feature': 2,
    'left': {   'feature': 0,
                'left': 0,
                'right': {'feature': 3, 'left': 1, 'right': 0, 'value': 1},
                'value': 1},
    'right': {   'feature': 0,
                 'left': 1,
                 'right': {'feature': 3, 'left': 1, 'right': 0, 'value': 1},
                 'value': 2},
    'value': 1}
accuracy 0.6666666666666666
precision: 0.75
recall: 0.75
f1measure 0.6666666666666666


In [ ]:
# # for k fold data
# data = pd.read_csv('project3_dataset4.txt',sep='\t',header=None)

# #lc = LabelEncoder()
# data[0] = data[0].map( {'sunny':0, 'overcast':1, 'rain':2})
# data[1] = data[1].map({'hot':0,'mild':1,'cool':2})
# data[2] = data[2].map({'high':0,'normal':1})
# data[3]= data[3].map({'weak':0,'strong':1})
# #data.iloc[:,4]= lc.fit_transform(data.iloc[:,4])
# labels = data.iloc[:,-1].values
# features = data.iloc[:,:-1].values

# kf = KFold(n_splits=10)
# precision = np.zeros(10)
# recall =  np.zeros(10)
# fscore =  np.zeros(10)
# accuracy =  np.zeros(10)
# i = 0
# for train_index, test_index in kf.split(features):
#     X_train,X_test = features[train_index],features[test_index]
#     y_train , y_test = labels[train_index], labels[test_index]
#     d = 4
#     y_pred = decision_tree(X_train, y_train, X_test,d)
#     p,r,f,s = score(y_test,y_pred)
#     a = accuracy_score(y_test,y_pred)
#     precision[i],recall[i],fscore[i],accuracy[i]= p.mean(),r.mean(),f.mean(),a
#     i +=1
    
    
# print('precision:',precision.mean())
# print('recall:',recall.mean())
# print('f1measure',fscore.mean())
# print('accuracy',accuracy.mean())

In [ ]:
data.head()

In [27]:
data

,0,1,2,3,4
0,0,0,0,0,0
1,0,0,0,1,0
2,1,0,0,0,1
3,2,1,0,0,1
4,2,2,1,0,1
5,2,2,1,1,0
6,1,2,1,1,1
7,0,1,0,0,0
8,0,2,1,0,1
9,2,1,1,0,1
